In [1]:
###### load modules
from Module_diagnostic_functions import *
from Module_propagation_functions import *
from scipy.optimize import curve_fit
from scipy import interpolate


In [2]:
bl_name = 'HRM'
var_t = 400
err_E = 0
err_f0 = 0
err_f1 = 0

cname = 'C1'
err_delta = 0    # [rad]
err_delta = 0
err_miscut = 0.

if_log = 1
if_slice = 0
if_close = 0

job_num = 0

err_name = '_delta_C1'
err_val_name = '_{}urad'.format(round(err_delta,3))


In [3]:
##### sampling parameters
sigT = var_t * 1e-15/2.355
E = 9481.
d_slit = 7e-6

t_res =sigT*2.355/10              # time sampling resolution [s]
ev_window = 4/t_res *1e-15        # total energy window [eV]
ev_res = min(ev_window/400, 1e-3) # energy sampling resolution [eV]


range_x = 4e-3; range_y = 4e-3
nx = 512; ny = 8; nz = 2*round(ev_window/ev_res/2)

x_res = range_x/nx
y_res = range_y/ny
t_window = t_res*nz          # total time window [s]

pulseRange = int(t_window/sigT)
factor = -1 # factor = 0.5

print('nx, ny, nz: {}'.format([nx, ny, nz]))
print('x resolution/range: {}/ {}um'.format(round(x_res*1e6,2), round(range_x*1e6,2)))
print('y resolution/range: {}/ {}um'.format(round(y_res*1e6,2), round(range_y*1e6,2)))
print('time resolution/range: {}/ {}fs'.format(round(t_res*1e15,2), round(t_window*1e15,2)))
print('energy resolution/range: {}/ {}meV'.format(round(ev_res*1e3,2), round(ev_window*1e3,2)))


nx, ny, nz: [512, 8, 400]
x resolution/range: 7.81/ 4000.0um
y resolution/range: 500.0/ 4000.0um
time resolution/range: 40.0/ 16000.0fs
energy resolution/range: 0.25/ 100.0meV


In [4]:
def calc_crystal_orientation(thetaB, ang_asym, ang_dif_pl, delta=0.):
    '''calculates the crystal orientation, dif_pl defines the direction of reflected beam in the incident frame
        ang_dif_pl: [0 to +y, pi/2 to -x, pi to -y, -pi/2 to +x], in betweeen values are allowed
            Note: Oleg's "+x" axis is actually the "-x" axis, so pi/2 corresponds to horizontal reflection to the left
        delta: crystal alignment error, >0 turns the normal vector toward the reflected beam
        tv represents direction of crystal surface
        nv represents direction of crystal surface normal
    '''
    ang_incidence = thetaB-ang_asym
    nv = np.array([0, np.cos(ang_incidence), -np.sin(ang_incidence)])
    tv = np.array([0, np.sin(ang_incidence), np.cos(ang_incidence)])
    rot_mat = calc_rot_mat_x(delta)
    nv = np.dot(rot_mat, nv)
    tv = np.dot(rot_mat, tv)
    rot_mat = calc_rot_mat_z(ang_dif_pl)
    nv_rot = np.dot(rot_mat, nv)
    tv_rot = np.dot(rot_mat, tv)
    return nv_rot, tv_rot, ang_incidence

In [5]:
##### general lens calculations
name_CRL = ['CRL0', 'CRL1', 'CRL2']
fCRL_list = [290, 10, 10]
nCRL_list = [1, 1, 1]

##### general crystal calculations
# Si 220
Si220 = srwlib.SRWLOptCryst(_d_sp=1.9201374688016222, _psi0r=-1.0873035035585694e-05, _psi0i=1.8438837339536554e-07,
                         _psi_hr=-6.610167427198717e-06, _psi_hi=1.7824173540780476e-07,
                         _psi_hbr=-6.610167427198717e-06, _psi_hbi=1.7824173540780476e-07,
                         _tc=0.01, _ang_as=0)

# Si 440
Si440 = srwlib.SRWLOptCryst(_d_sp=0.9600687344008111, _psi0r=-1.0873035035585694e-05, _psi0i=1.8438837339536554e-07,
                         _psi_hr=-4.181686438547451e-06, _psi_hi=1.6100412693351052e-07,
                         _psi_hbr=-4.181686438547451e-06, _psi_hbi=1.6100412693351052e-07,
                         _tc=0.01, _ang_as=0)

thetaB220 = Si220.get_ang_inc(_e=E)
thetaB440 = Si440.get_ang_inc(_e=E)


In [6]:
##### beamline crystal calculations
''' input parameters '''
name_HRM = ['C1', 'C2', 'C3', 'C4']
thetaB_list_HRM = np.array([thetaB440, thetaB440, thetaB440, thetaB440])
ang_asym_list_HRM = np.deg2rad([0, -29.5, 0, 29.5])
ang_dif_pl_list_HRM = np.array([np.pi/2, -np.pi/2, -np.pi/2, np.pi/2])

''' var_param '''
nv_list_HRM = []; tv_list_HRM = []; ang_in_HRM = []
for i in range(len(name_HRM)):
    thetaB = thetaB_list_HRM[i]
    ang_asym = ang_asym_list_HRM[i]
    ang_dif_pl = ang_dif_pl_list_HRM[i]
    nv, tv, ang_in = calc_crystal_orientation(thetaB, ang_asym, ang_dif_pl)
    nv_list_HRM.append(nv)
    tv_list_HRM.append(tv)
    ang_in_HRM.append(ang_in)
    
print('HRM var_param\n',
      'nv: \n{}\n'.format(np.round(nv_list_HRM,3)),
      'tv: \n{}\n'.format(np.round(tv_list_HRM,3)),
      'incident angle: \n{}\n'.format(np.round(ang_in_HRM,3)))

''' scale factors '''
b_factor_HRM = calc_b_factor(thetaB_list_HRM,ang_asym_list_HRM).max()
x_scaling_HRM = calc_scale_factor(b_factor_HRM)
t_stretching_HRM = calc_t_stretching(thetaB_list_HRM,ang_asym_list_HRM, range_x=range_x).max()
z_scaling_HRM = np.max([np.round(t_stretching_HRM/t_window),1.])

print('HRM scaling\n',
      'b factor: {}\n'.format(round(b_factor_HRM,2)),
      't stretching: {}fs\n'.format(round(t_stretching_HRM*1e15,2)),
      'x scale: {}, z scale: {}\n'.format(x_scaling_HRM, z_scaling_HRM))



HRM var_param
 nv: 
[[-0.732  0.    -0.681]
 [ 0.302  0.    -0.953]
 [ 0.732  0.    -0.681]
 [-0.973  0.    -0.232]]
 tv: 
[[-0.681  0.     0.732]
 [ 0.953  0.     0.302]
 [ 0.681  0.     0.732]
 [-0.232  0.     0.973]]
 incident angle: 
[0.749 1.264 0.749 0.234]

HRM scaling
 b factor: 4.11
 t stretching: 9381.0fs
 x scale: 4, z scale: 1.0



In [7]:
varParam = srwl_bl.srwl_uti_ext_options([
    ['name', 's', 'full_beamline', 'simulation name'],

#---Data Folder
    ['fdir', 's', '', 'folder (directory) name for reading-in input and saving output data files'],


    ['gbm_x', 'f', 0.0, 'average horizontal coordinates of waist [m]'],
    ['gbm_y', 'f', 0.0, 'average vertical coordinates of waist [m]'],
    ['gbm_z', 'f', 0.0, 'average longitudinal coordinate of waist [m]'],
    ['gbm_xp', 'f', 0.0, 'average horizontal angle at waist [rad]'],
    ['gbm_yp', 'f', 0.0, 'average verical angle at waist [rad]'],
    ['gbm_ave', 'f', E+err_E, 'average photon energy [eV]'],
    ['gbm_pen', 'f', 0.001, 'energy per pulse [J]'],
    ['gbm_rep', 'f', 1, 'rep. rate [Hz]'],
    ['gbm_pol', 'f', 2, 'polarization 1- lin. hor., 2- lin. vert., 3- lin. 45 deg., 4- lin.135 deg., 5- circ. right, 6- circ. left'],
    ['gbm_sx', 'f', 9.787229999999999e-06, 'rms beam size vs horizontal position [m] at waist (for intensity)'],
    ['gbm_sy', 'f', 9.787229999999999e-06, 'rms beam size vs vertical position [m] at waist (for intensity)'],
    ['gbm_st', 'f', sigT, 'rms pulse duration [s] (for intensity)'],
    ['gbm_mx', 'f', 0, 'transverse Gauss-Hermite mode order in horizontal direction'],
    ['gbm_my', 'f', 0, 'transverse Gauss-Hermite mode order in vertical direction'],
    ['gbm_ca', 's', 'c', 'treat _sigX, _sigY as sizes in [m] in coordinate representation (_presCA="c") or as angular divergences in [rad] in angular representation (_presCA="a")'],
    ['gbm_ft', 's', 't', 'treat _sigT as pulse duration in [s] in time domain/representation (_presFT="t") or as bandwidth in [eV] in frequency domain/representation (_presFT="f")'],

#---Calculation Types
    #Single-Electron Intensity distribution vs horizontal and vertical position
    ['si', '', '', 'calculate single-e intensity distribution (without wavefront propagation through a beamline) vs horizontal and vertical position', 'store_true'],
    #Single-Electron Wavefront Propagation
    ['ws', '', '', 'calculate single-electron (/ fully coherent) wavefront propagation', 'store_true'],
    #Multi-Electron (partially-coherent) Wavefront Propagation
    ['wm', '', '', 'calculate multi-electron (/ partially coherent) wavefront propagation', 'store_true'],

    ['w_e', 'f', -pulseRange*sigT/2, 'photon energy [eV] for calculation of intensity distribution vs horizontal and vertical position'],
    ['w_ef', 'f', pulseRange*sigT/2, 'final photon energy [eV] for calculation of intensity distribution vs horizontal and vertical position'],
    ['w_ne', 'i', nz, 'number of points vs photon energy for calculation of intensity distribution'],
    ['w_x', 'f', 0.0, 'central horizontal position [m] for calculation of intensity distribution'],
    ['w_rx', 'f', range_x, 'range of horizontal position [m] for calculation of intensity distribution'],
    ['w_nx', 'i', nx, 'number of points vs horizontal position for calculation of intensity distribution'],
    ['w_y', 'f', 0.0, 'central vertical position [m] for calculation of intensity distribution vs horizontal and vertical position'],
    ['w_ry', 'f', range_y, 'range of vertical position [m] for calculation of intensity distribution vs horizontal and vertical position'],
    ['w_ny', 'i', ny, 'number of points vs vertical position for calculation of intensity distribution'],
    ['w_smpf', 'f', factor, 'sampling factor for calculation of intensity distribution vs horizontal and vertical position'],
    ['w_meth', 'i', 2, 'method to use for calculation of intensity distribution vs horizontal and vertical position: 0- "manual", 1- "auto-undulator", 2- "auto-wiggler"'],
    ['w_prec', 'f', 0.01, 'relative precision for calculation of intensity distribution vs horizontal and vertical position'],

    ['w_ft', 's', 't', 'presentation/domain: "f"- frequency (photon energy), "t"- time'],

    ['w_u', 'i', 2, 'electric field units: 0- arbitrary, 1- sqrt(Phot/s/0.1%bw/mm^2), 2- sqrt(J/eV/mm^2) or sqrt(W/mm^2), depending on representation (freq. or time)'],
    ['si_pol', 'i', 6, 'polarization component to extract after calculation of intensity distribution: 0- Linear Horizontal, 1- Linear Vertical, 2- Linear 45 degrees, 3- Linear 135 degrees, 4- Circular Right, 5- Circular Left, 6- Total'],
    ['si_type', 'i', 0, 'type of a characteristic to be extracted after calculation of intensity distribution: 0- Single-Electron Intensity, 1- Multi-Electron Intensity, 2- Single-Electron Flux, 3- Multi-Electron Flux, 4- Single-Electron Radiation Phase, 5- Re(E): Real part of Single-Electron Electric Field, 6- Im(E): Imaginary part of Single-Electron Electric Field, 7- Single-Electron Intensity, integrated over Time or Photon Energy'],
    
    ['si_fn', 's', 'res_int_se.dat', 'file name for saving calculated single-e intensity distribution (without wavefront propagation through a beamline) vs horizontal and vertical position'],
    ['si_pl', 's', '', 'plot the input intensity distributions in graph(s): ""- dont plot, "x"- vs horizontal position, "y"- vs vertical position, "xy"- vs horizontal and vertical position'],
    ['ws_fni', 's', 'res_int_pr_se.dat', 'file name for saving propagated single-e intensity distribution vs horizontal and vertical position'],
    ['ws_pl', 's', '', 'plot the resulting intensity distributions in graph(s): ""- dont plot, "x"- vs horizontal position, "y"- vs vertical position, "xy"- vs horizontal and vertical position'],

    #to add options
    ['op_r', 'f', 290.0, 'longitudinal position of the first optical element [m]'],
    # Former appParam:
    ['rs_type', 's', 'g', 'source type, (u) idealized undulator, (t), tabulated undulator, (m) multipole, (g) gaussian beam'],
    
#---Propagation parameters
#                                   [0][1] [2] [3][4] [5]  [6]  [7]  [8]  [9] [10] [11]
    ['op_CRL_pp', 'f',              [0, 0, 1.0, 0, 0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0], 'CRL'],
    ['op_CRL_C1_pp', 'f',           [0, 0, 1.0, 1, 0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0], 'CRL_C1'],
    ['op_C1_pp', 'f',               [0, 0, 1.0, 0, 0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0], 'C1'],
    ['op_fin_pp', 'f',              [0, 0, 1.0, 0, 0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0], 'final post-propagation (resize) parameters'],

    #[ 0]: Auto-Resize (1) or not (0) Before propagation
    #[ 1]: Auto-Resize (1) or not (0) After propagation
    #[ 2]: Relative Precision for propagation with Auto-Resizing (1. is nominal)
    #[ 3]: Allow (1) or not (0) for semi-analytical treatment of the quadratic (leading) phase terms at the propagation
    #[ 4]: Do any Resizing on Fourier side, using FFT, (1) or not (0)
    #[ 5]: Horizontal Range modification factor at Resizing (1. means no modification)
    #[ 6]: Horizontal Resolution modification factor at Resizing
    #[ 7]: Vertical Range modification factor at Resizing
    #[ 8]: Vertical Resolution modification factor at Resizing
    #[ 9]: Type of wavefront Shift before Resizing (not yet implemented)
    #[10]: New Horizontal wavefront Center position after Shift (not yet implemented)
    #[11]: New Vertical wavefront Center position after Shift (not yet implemented)
    #[12]: Optional: Orientation of the Output Optical Axis vector in the Incident Beam Frame: Horizontal Coordinate
    #[13]: Optional: Orientation of the Output Optical Axis vector in the Incident Beam Frame: Vertical Coordinate
    #[14]: Optional: Orientation of the Output Optical Axis vector in the Incident Beam Frame: Longitudinal Coordinate
    #[15]: Optional: Orientation of the Horizontal Base vector of the Output Frame in the Incident Beam Frame: Horizontal Coordinate
    #[16]: Optional: Orientation of the Horizontal Base vector of the Output Frame in the Incident Beam Frame: Vertical Coordinate
])


In [8]:
## set optics
def set_optics_CRL0_before_C1(v=None):
    el = []
    pp = []
    names = ['CRL','CRL_C1']
    for el_name in names:
        if el_name == 'CRL':
            # CRL: crl 290.0m
            el.append(srwlib.srwl_opt_setup_CRL(
                _foc_plane=1,
                _delta=3.791135e-06,
                _atten_len=0.008387,
                _shape=1,
                _apert_h=range_x,
                _apert_v=range_y,
                _r_min=rCRL(fCRL_list[0], nCRL_list[0]),
                _n= nCRL_list[0],
                _wall_thick=5e-5,
                _xc=0,
                _yc=0,
            ))
            pp.append(v.op_CRL_pp)
        elif el_name == 'CRL_C1':
            # CRL_C1: drift 290.0m
            el.append(srwlib.SRWLOptD(
                _L=10.0,
            ))
            pp.append(v.op_CRL_C1_pp)
    return srwlib.SRWLOptC(el, pp)

##### introduce errors
def set_optics_C1(v=None, err_delta=0.):
    nv, tv, ang_in = calc_crystal_orientation(thetaB, ang_asym, ang_dif_pl, delta=err_delta)
    
    el = []
    pp = []
    names = ['C1']
    for el_name in names:
        if el_name == 'C1':
            # C1: crystal 300.0m
            crystal = srwlib.SRWLOptCryst(
                _d_sp=Si440.dSp,
                _psi0r=Si440.psi0r,
                _psi0i=Si440.psi0i,
                _psi_hr=Si440.psiHr,
                _psi_hi=Si440.psiHi,
                _psi_hbr=Si440.psiHbr,
                _psi_hbi=Si440.psiHbi,
                _tc=Si440.tc,
                _ang_as=ang_asym_list_HRM[0],
                _nvx=nv[0],
                _nvy=nv[1],
                _nvz=nv[2],
                _tvx=tv[0],
                _tvy=tv[1],
                _uc=Si440.uc,
                _e_avg=E,
                _ang_roll=ang_dif_pl_list_HRM[0]
            )
            el.append(crystal)
            pp.append(v.op_C1_pp)
    return srwlib.SRWLOptC(el, pp)


In [9]:
####### I/O
## output directories
dir_output = 'output/'
dir_bl_out = dir_output+'{}/'.format(bl_name)    # HHLM, HRM, HHLM_HRM
dir_dur_out = dir_bl_out+'{}fs{}/'.format(round(sigT*2.355*1e15,1), err_name)    # pulse duration (100.0fs/)
dir_plot = dir_dur_out+'{}fs_{}meV_job{}{}/'.format(round(t_window*1e15,1), round(ev_window*1e3,1), job_num, err_val_name)    # sampling range + error value

for dir_name in [dir_output, dir_bl_out, dir_dur_out, dir_plot]:
    mkdir(dir_name); # print('directory created: {}'.format(dir_name))

print('job {}\n    directories created'.format(job_num))

## script variables
if if_close == 0: i_start = 0

z_scaling = z_scaling_HRM
if if_close == 1: i_start = 0


job 0
    directories created


In [10]:
def get_intensity_slice(_wfr, axis='x'):
    nx, ny, nz = get_dimension(_wfr)
    intensity = get_intensity(_wfr, domain='t', polarization='total')[:,:,0]
    if axis == 'x':
        intensity_slice = intensity[int(ny/2)]
        intensity_projection = intensity.sum(axis=0)
        axis_sp = get_axis_sp(_wfr)[0]
    elif axis == 'y':
        intensity_slice = intensity[:,int(nx/2)]
        intensity_projection = intensity.sum(axis=1)
        axis_sp = get_axis_sp(_wfr)[1]
    return axis_sp, intensity_slice, intensity_projection

def main_HRM(err_delta_list, varParam, z_scaling=10.0, dir_plot=None, if_log=1, if_slice=1, i_start=0):
    time_stamp=time()
    
    v = srwl_bl.srwl_uti_parse_options(varParam, use_sys_argv=True)
    v.si = True
    srwl_bl.SRWLBeamline(_name=v.name).calc_all(v)

    # incident beam
    wfr = v.w_res; i_plot = 1
    plot_wfr_diagnostic(wfr, label='input', dir_plot=dir_plot, i=i_plot, if_log=if_log, if_slice=if_slice, i_start=i_start)
    srwlpy.SetRepresElecField(wfr, 'f')

    t0 = time(); print('Resizing in frequency domain: ', end='')
    srwlpy.ResizeElecField(wfr, 'f', [0, 1., z_scaling]); print('done in', round(time() - t0, 3), 's')

    # diagnostics - input
    diagnostics, diagnostics_names = diagnose_input(wfr)

    # crystal 1
    label = 'before C1'; print('Propagating to {}: '.format(label), end=''); i_plot+=1; t0 = time()
    bl = set_optics_CRL0_before_C1(v); srwlpy.PropagElecField(wfr, bl); print('done in', round(time() - t0, 3), 's\n')
    plot_wfr_diagnostic(wfr, label=label, dir_plot=dir_plot, i=i_plot, if_log=if_log, if_slice=if_slice, i_start=i_start)
    
    wfr_back = deepcopy(wfr)
    for j, err_delta in enumerate(err_delta_list):
        wfr = deepcopy(wfr_back)
        label = 'after C1'; print('Propagating to {}: '.format(label), end=''); i_plot+=1; t0 = time()
        bl = set_optics_C1(v, err_delta=err_delta); srwlpy.PropagElecField(wfr, bl); print('done in', round(time() - t0, 3), 's\n')
        plot_wfr_diagnostic(wfr, label=label, dir_plot=dir_plot, i=i_plot, if_log=if_log, if_slice=if_slice, i_start=i_start)
        axis_x, slice_x, projection_x = get_intensity_slice(wfr, axis='x')
        axis_y, slice_y, projection_y = get_intensity_slice(wfr, axis='y')
        if j == 0:
            # expand axis for interpolation
            x_new = np.linspace(axis_x.min(), axis_x.max(), axis_x.size * 5)
            y_new = np.linspace(axis_y.min(), axis_y.max(), axis_y.size * 5)
            slice_x_list = np.zeros((err_delta_list.size, x_new.size))
            slice_y_list = np.zeros((err_delta_list.size, y_new.size))
            
        fx = interpolate.interp1d(axis_x, slice_x, kind='linear', bounds_error=False, fill_value=0); slice_x_list[j] = fx(x_new)
        fy = interpolate.interp1d(axis_y, slice_y, kind='linear', bounds_error=False, fill_value=0); slice_y_list[j] = fy(y_new)
        
        plt.close('all')
    print('\n\neverything lasted: {}s'.format(round(time()-time_stamp,2)))
    return x_new, slice_x_list, y_new, slice_y_list


In [11]:
err_delta_list = np.deg2rad(np.linspace(-10, 10, 51))

if __name__ == '__main__':
    axis_x, slice_x_list, axis_y, slice_y_list = main_HRM(err_delta_list, varParam, z_scaling=1, dir_plot=dir_plot, if_log=1, if_slice=0, i_start=0)


Gaussian beam electric field calculation ... completed (lasted 0.137 s)
Extracting intensity and saving it to a file ... completed (lasted 2.1 s)


C:\Users\nanw0\anaconda3\envs\SRW\lib\site-packages\scipy\optimize\minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


del_t*del_E/h: 0.86497
plotting input,
 [nx, ny, nz]: [512, 8, 400]
 x range: 4000.0um
 t range: 15966.03fs
 E range: 103.09meV

plot lasted 1.86s

Resizing in frequency domain: done in 0.0 s
fitting for beam diagnostics
done in 0.092 s

Propagating to before C1: Optical Element Setup: CRL Focal Length: 290.0 m


C:\Users\nanw0\GoogleDrive\SLAC\SLAC-Diling\11. 21 Winter\DXS optics\Crystal_error_test\Module_diagnostic_functions.py:556: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if values == 'Fit failed':


done in 1.939 s

del_t*del_E/h: 0.86531
plotting before C1,
 [nx, ny, nz]: [512, 8, 400]
 x range: 4000.0um
 t range: 15966.03fs
 E range: 103.09meV

plot lasted 1.5s

Propagating to after C1: done in 1.013 s

Fit failed. Using second moment for width.
del_t*del_E/h: 0.86294
plotting after C1,
 [nx, ny, nz]: [512, 8, 400]
 x range: 4000.0um
 t range: 15966.03fs
 E range: 103.09meV

plot lasted 1.8s

Propagating to after C1: done in 0.999 s

Fit failed. Using second moment for width.
del_t*del_E/h: 0.86294
plotting after C1,
 [nx, ny, nz]: [512, 8, 400]
 x range: 4000.0um
 t range: 15966.03fs
 E range: 103.09meV

plot lasted 1.77s

Propagating to after C1: done in 1.019 s

Fit failed. Using second moment for width.
del_t*del_E/h: 0.86294
plotting after C1,
 [nx, ny, nz]: [512, 8, 400]
 x range: 4000.0um
 t range: 15966.03fs
 E range: 103.09meV

plot lasted 1.77s

Propagating to after C1: done in 1.005 s

Fit failed. Using second moment for width.
del_t*del_E/h: 0.86294
plotting after C1

Fit failed. Using second moment for width.
del_t*del_E/h: 0.86294
plotting after C1,
 [nx, ny, nz]: [512, 8, 400]
 x range: 4000.0um
 t range: 15966.03fs
 E range: 103.09meV

plot lasted 1.69s

Propagating to after C1: done in 0.965 s

Fit failed. Using second moment for width.
del_t*del_E/h: 0.86294
plotting after C1,
 [nx, ny, nz]: [512, 8, 400]
 x range: 4000.0um
 t range: 15966.03fs
 E range: 103.09meV

plot lasted 1.69s

Propagating to after C1: done in 0.968 s

Fit failed. Using second moment for width.
del_t*del_E/h: 0.86294
plotting after C1,
 [nx, ny, nz]: [512, 8, 400]
 x range: 4000.0um
 t range: 15966.03fs
 E range: 103.09meV

plot lasted 1.68s

Propagating to after C1: done in 0.967 s

Fit failed. Using second moment for width.
del_t*del_E/h: 0.86294
plotting after C1,
 [nx, ny, nz]: [512, 8, 400]
 x range: 4000.0um
 t range: 15966.03fs
 E range: 103.09meV

plot lasted 1.7s

Propagating to after C1: done in 0.976 s

Fit failed. Using second moment for width.
del_t*del_E/h: